# Standard Name Table

A so-called "standard name table" defines "standard names", which is a concept used by the [CF Convention](https://cfconventions.org/).

Those standard names are used to define the meaning of a numerical variable in files (typically netCDF4 files).

With this library, we can describe a standard name table using JSON-LD. **Note**, that only a simplified version of the original CF Conventions is modelled!

This notebook walks you through the main steps of building such a table yourself using Python:

In [1]:
import ssnolib
from ssnolib.namespace import SSNO
from ssnolib.prov import Person

Let's start by instantiate a table class. We add a title and a responsible person:

In [2]:
snt = ssnolib.StandardNameTable(
    title='SNT from scratch',
    creator=Person(orcidID="https://orcid.org/0000-0001-8729-0482")
)

Let's add some standard names to the table:

In [3]:
snt.standardNames = [
    ssnolib.StandardName(
        standard_name="air_density",
        description="The density of air",
        canonicalUnits="kg/m^3"
    ),
    ssnolib.StandardName(
        standard_name="coordinate",
        description="The spatial coordinate vector.",
        canonicalUnits="m"
    ),
    ssnolib.StandardName(
        standard_name="velocity",
        description="The velocity vector of an object or fluid.",
        canonicalUnits="m/s"
    )
]

So far we only have two standard names. We can define modification rules, to build new, verified standard names. For example, "x_velocity" would be a reasonable new standard name for the table.

So let's define such a modification rule. We call it a `Qualification`. The one we would like to define should be used directly of an already existing standard name, e.g. "SSNO:AnyStandardName":

In [4]:
component = ssnolib.Qualification(
    name="component",
    hasValidValues=["x", "y", "z"],
    description="The component of a vector",
    before=SSNO.AnyStandardName
)

transformation = ssnolib.Transformation(
    name="C_derivative_of_X",
    description="derivative of X with respect to distance in the component direction, which may be x, y or z.",
    altersUnit="[X]/[C]",
    hasCharacter=[
        ssnolib.Character(character="X", associatedWith=SSNO.AnyStandardName),
        ssnolib.Character(character="C", associatedWith=component.id),
    ]
)

Add it to the SNT:

In [5]:
snt.hasModifier = [component, transformation]

We can check standard name strings, whether they apply to the modification rule:

In [6]:
snt.verify_name("vertical_velocity")

False

In [7]:
snt.verify_name("x_velocity")

True

In [8]:
snt.verify_name("x_component")

False

In [9]:
snt.add_new_standard_name("x_coordinate", verify=True) # verify=False will just add the standard name and interpret it as a core standard name

StandardName(id=_:N082b0aa95eec40c9a1bf83d308818a98, canonicalUnits=http://qudt.org/vocab/unit/UNITLESS, standardName=x_coordinate, description=N.A)

We can add more qualifications and also transformations. More on this in a separate chapter further below.

Now, let's export the current version of the standard name table to JSON-LD:

In [10]:
print(snt.model_dump_jsonld())

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "dcat": "http://www.w3.org/ns/dcat#",
        "dcterms": "http://purl.org/dc/terms/",
        "prov": "http://www.w3.org/ns/prov#",
        "ssno": "https://matthiasprobst.github.io/ssno#",
        "foaf": "http://xmlns.com/foaf/0.1/",
        "m4i": "http://w3id.org/nfdi4ing/metadata4ing#",
        "schema": "http://schema.org/",
        "skos": "http://www.w3.org/2004/02/skos/core#"
    },
    "@type": "ssno:StandardNameTable",
    "dcterms:title": "SNT from scratch",
    "dcterms:creator": {
        "@type": "prov:Person",
        "orcidID": "https://orcid.org/0000-0001-8729-0482",
        "@id": "_:Nf1f8defb731d4c239391b017ae0da3a5"
    },
    "ssno:standardNames": [
        {
            "@type": "ssno:StandardName",
            "ssno:canonicalUnits": "http://qudt.org/vocab/unit/KiloGM-PER-M3",
            "ssno:standardName": "air_density",
       

Transformation "component_derivative_of_X"

In [11]:
print(component.model_dump_jsonld())

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "ssno": "https://matthiasprobst.github.io/ssno#",
        "schema": "http://schema.org/",
        "dcterms": "http://purl.org/dc/terms/"
    },
    "@type": "ssno:Qualification",
    "schema:name": "component",
    "dcterms:description": "The component of a vector",
    "ssno:before": "https://matthiasprobst.github.io/ssno#AnyStandardName",
    "ssno:hasValidValues": [
        {
            "@type": "ssno:AnnotatedValue",
            "ssno:value": "x",
            "ssno:annotation": "No description available.",
            "@id": "_:N1fdd207c9acd49e09ef350e7c8dae3a6"
        },
        {
            "@type": "ssno:AnnotatedValue",
            "ssno:value": "y",
            "ssno:annotation": "No description available.",
            "@id": "_:Nd2d2c59628bd424a86796c40f9f1a760"
        },
        {
            "@type": "ssno:AnnotatedValue",
            "s

In [12]:
transformation = ssnolib.Transformation(
    name="C_derivative_of_X",
    description="derivative of X with respect to distance in the component direction, which may be x, y or z.",
    altersUnit="[X]/[C]",
    hasCharacter=[
        ssnolib.Character(character="X", associatedWith=SSNO.AnyStandardName),
        ssnolib.Character(character="C", associatedWith=component.id),
    ]
)

In [13]:
print(transformation.model_dump_jsonld())

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "ssno": "https://matthiasprobst.github.io/ssno#",
        "schema": "http://schema.org/",
        "dcterms": "http://purl.org/dc/terms/"
    },
    "@type": "ssno:Transformation",
    "schema:name": "C_derivative_of_X",
    "dcterms:description": "derivative of X with respect to distance in the component direction, which may be x, y or z.",
    "ssno:unitModificationRule": "[X]/[C]",
    "ssno:hasCharacter": [
        {
            "@type": "ssno:Character",
            "character": "X",
            "ssno:associatedWith": "https://matthiasprobst.github.io/ssno#AnyStandardName",
            "@id": "_:Ne2eef7645dcf4d729dc53a945b377c2b"
        },
        {
            "@type": "ssno:Character",
            "character": "C",
            "ssno:associatedWith": "_:N072f2479020144fbb18ce5fc04efdd01",
            "@id": "_:Ne537dacdda3c4ca78335864301515457"
   

In [14]:
snt.hasModifier = [component, ]

In [15]:
snt.standard_names = [ssnolib.StandardName(standardName="velocity", canonicalUnits="m/s", description="Velocity"),]

In [16]:
snt.standard_names[0]

StandardName(id=_:N17f5049d3c184353a0140b70d525a2d1, canonicalUnits=http://qudt.org/vocab/unit/M-PER-SEC, standardName=velocity, description=Velocity)

In [17]:
snt.verify("x_velocity")

AttributeError: 'str' object has no attribute 'standardName'

In [ ]:
snt.get_qualification_rule_as_string()

Dump the objects as JSON-LD:

In [ ]:
print(snt.model_dump_jsonld())

In [ ]:
import re

# Define the regex pattern
pattern = r"^((x|y|z)_)?velocity(_in_(air|water|oil))?$"

In [ ]:
verify_standard_name_with_regex("velocity_in_oil")

In [ ]:
orig_pattern = r"(toa|tropopause|surface)?_?((x|y|z)_)?_?velocity$"

In [ ]:
pattern = orig_pattern.replace("standard_name", "velocity")
pattern

In [ ]:
orig_pattern = "^(toa|tropopause|surface)?_?(upward|downward|northward|southward|eastward|westward|x|y)?_?standard_name_?(at_(adiabatic_condensation_level|cloud_top|convective_cloud_top|cloud_base|convective_cloud_base|freezing_level|ground_level|maximum_wind_speed_level|sea_floor|sea_ice_base|sea_level|top_of_atmosphere_boundary_layer|top_of_atmosphere_model|top_of_dry_convection))$"

In [ ]:
"^(toa|tropopause|surface)?_?(upward|downward|northward|southward|eastward|westward|x|y)?_?velocity_?(at_adiabatic_condensation_level|at_cloud_top)?$"

In [ ]:
pattern = orig_pattern.replace("standard_name", "velocity")
pattern

In [ ]:
def verify_standard_name_with_regex(standard_name):
    if re.match(pattern, standard_name):
        return f"'{standard_name}' is a valid standard name."
    else:
        return f"'{standard_name}' is NOT a valid standard name."

In [ ]:
pattern

In [ ]:
verify_standard_name_with_regex("toa_velocity_at_adiabatic_condensation_level")

In [ ]:
import re

# Define the refined regex pattern
# pattern = r'^(?:toa|tropopause|surface)?(?:_?(?:upward|downward|northward|southward|eastward|westward|x|y))?_?air_pressure(?:_at_(adiabatic_condensation_level|cloud_top|convective_cloud_top|cloud_base|convective_cloud_base|freezing_level|ground_level|maximum_wind_speed_level|sea_floor|sea_ice_base|sea_level|top_of_atmosphere_boundary_layer|top_of_atmosphere_model|top_of_dry_convection))?(?:_in_(air|atmosphere_boundary_layer|mesosphere|sea_ice|sea_water|soil|soil_water|stratosphere|thermosphere|troposphere))?(?:_due_to_(advection|convection|deep_convection|diabatic_processes|diffusion|dry_convection|gravity_wave_drag|gyre|isostatic_adjustment|large_scale_precipitation|longwave_heating|moist_convection|overturning|shallow_convection|shortwave_heating|thermodynamics))?(?:_assuming_(clear_sky|deep_snow|no_snow))?$'
pattern = r'^(?:toa|tropopause|surface)?_?(?:upward|downward|northward|southward|eastward|westward|x|y)?_?standard_name(?:_(?:at_adiabatic_condensation_level|at_cloud_top|at_convective_cloud_top|at_cloud_base|at_convective_cloud_base|at_freezing_level|at_ground_level|at_maximum_wind_speed_level|at_sea_floor|at_sea_ice_base|at_sea_level|at_top_of_atmosphere_boundary_layer|at_top_of_atmosphere_model|at_top_of_dry_convection))?$'
# Function to verify the standard name using regex
def verify_standard_name(standard_name):
    if re.match(pattern, standard_name):
        return f"'{standard_name}' is a valid standard name."
    else:
        return f"'{standard_name}' is not a valid standard name."

# Test cases
print(verify_standard_name("toa_upward_air_pressure_at_adiabatic_condensation_level"))  # Expected: valid
print(verify_standard_name("toa_upward_air_pressure_at_cloud_top"))  # Expected: valid
print(verify_standard_name("surface_air_pressure_in_sea_water_due_to_diffusion"))  # Expected: valid
print(verify_standard_name("air_pressure"))  # Expected: valid (no qualifiers)
